<a href="https://colab.research.google.com/github/IgnacioMurubeCrego/practica5ArqCompu/blob/main/practica5_ArqCompu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [2]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-f5uontjk
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-f5uontjk
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 28f872a2f99a1b201bcd0db14fdbc5a496b9bfd7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.2.1-py3-none-any.whl size=10742 sha256=fdb30b442a03a2b978d94f242f4f3839c3b517111d4c5bcc31496f40b5aacbba
  Stored in directory: /tmp/pip-ephem-wheel-cache-lch8r_k8/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built nvcc4jupyter


In [3]:
%load_ext nvcc4jupyter

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmpodbxn7zu".


In [4]:
%%cuda
#include <stdio.h>

__global__ void hello(){
    printf("Hello World from block: %u, thread: %u\n", blockIdx.x, threadIdx.x);
}

int main(){
    hello<<<2, 3>>>();
    cudaDeviceSynchronize();
}

Hello World from block: 0, thread: 0
Hello World from block: 0, thread: 1
Hello World from block: 0, thread: 2
Hello World from block: 1, thread: 0
Hello World from block: 1, thread: 1
Hello World from block: 1, thread: 2



### Ejemplo B :  duplicar un valor con CUDA


In [5]:
%%cuda
#include <iostream>
#include <cuda_runtime.h>

// DEFINICIÓN KERNEL CUDA

__global__ void duplicarValor(int *valor_device)

{
  printf("Block [2%u], Thread [2%u]: valor_device=%d\n", blockIdx.x, threadIdx.x, *valor_device);
  *valor_device *= 2;
}

// FUNCIÓN DEL HOST

int main()
{

  // 1 - INICIALIZAR DATOS DEL HOST

  int valor_host = 5;

  // valor inicial en el host

  // 2 - DECLARAR Y ASIGNAR MEMORIA PARA EL DEVICE

  int *valor_device; // puntero al device

  cudaMalloc((void **)&valor_device, sizeof(int)); // asignar memoria para el device

  // 3 - TRANSFERIR DATOS DEL HOST AL DEVICE

  cudaMemcpy(valor_device, &valor_host, sizeof(int), cudaMemcpyHostToDevice); // transferir el valor del host al device

  // 4 - EJECUTAR EN UNO O MÁS KERNELS

  duplicarValor<<<1, 2>>>(valor_device); // lanzar el kernel

  cudaDeviceSynchronize(); // espera hasta que el kernel termine

  // 5 - TRANSFERIR DATOS DEL DEVICE AL HOST

  cudaMemcpy(&valor_host, valor_device, sizeof(int), cudaMemcpyDeviceToHost); // transferir el valor del device al host

  // 6 - LIBERAR RECURSOS

  cudaFree(valor_device); // liberar memoria del device

  // MOSTRAR RESULTADO

  std::cout << "Resultado: " << valor_host << std::endl;

  return 0;
}

Block [20], Thread [20]: valor_device=5
Block [20], Thread [21]: valor_device=5
Resultado: 10



### Ejemplo C : suma de vectores con CUDA


In [6]:
%%cuda
#include <iostream>
#include <cuda_runtime.h>
#define T_VEC 100

//DEFINICIÓN KERNEL CUDA

__global__ void sumaVectores(int* v1_device, int* v2_device, int*suma_device) {
  printf("Índice de hilo dentro del bloque: %d\n", threadIdx.x);
  printf("Índice del bloque: %d\n", blockIdx.x);
  printf("Tamaño del bloque (número de hilos): %d\n", blockDim.x);
  for (int i = 0; i < T_VEC; i++) {
    suma_device[i] = v1_device[i] + v2_device[i];
  }
}

//FUNCIÓN DEL HOST

int main() {

  //1 - INICIALIZAR DATOS DEL HOST

  int v1_host[T_VEC], v2_host[T_VEC], suma_host[T_VEC];
  for (int i = 0; i < T_VEC; i++) {
    v1_host[i] = 1;
    v2_host[i] = 2;
  }

  //2 - DECLARAR Y ASIGNAR MEMORIA PARA EL DEVICE

  int *v1_device, *v2_device, *suma_device; //punteros al device
  cudaMalloc((void**)&v1_device, sizeof(int) * T_VEC); //asignar memoria para el device
  cudaMalloc((void**)&v2_device, sizeof(int) * T_VEC);
  cudaMalloc((void**)&suma_device, sizeof(int) * T_VEC);

  //3 - TRANSFERIR DATOS DEL HOST AL DEVICE

  cudaMemcpy(v1_device, v1_host, sizeof(int) * T_VEC,cudaMemcpyHostToDevice); //transferir el valor del host al device
  cudaMemcpy(v2_device, v2_host, sizeof(int) * T_VEC,cudaMemcpyHostToDevice);
  //cudaMemcpy(suma_device, suma_host, sizeof(int) * T_VEC,cudaMemcpyHostToDevice); //Inutil

  //4 - EJECUTAR EN UNO O MÁS KERNELS

  sumaVectores<<<1, 1>>>(v1_device, v2_device, suma_device);
  //lanzar el kernel
  cudaDeviceSynchronize(); //espera hasta que el kernel termine

  //5 - TRANSFERIR DATOS DEL DEVICE AL HOST

  cudaMemcpy(suma_host, suma_device, sizeof(int) * T_VEC,
  cudaMemcpyDeviceToHost); //transferir el valor del device al host

  //6 - LIBERAR RECURSOS
  cudaFree(v1_device); //liberar memoria del device
  cudaFree(v2_device);
  cudaFree(suma_device);

  //MOSTRAR RESULTADO
  for (int i = 0; i < T_VEC; i++) {
    std::cout << suma_host[i] << " ";
  }
  std::cout << "\n";
 return 0;
}

Índice de hilo dentro del bloque: 0
Índice del bloque: 0
Tamaño del bloque (número de hilos): 1
3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 



#Ejercicio 1 (4 puntos)
Modificar el ejemplo de suma de vectores con CUDA para que se ejecute en un bloque de
25 hilos, dónde cada uno se encargue de sumar una parte de los vectores.

In [11]:
%%cuda
#include <iostream>
#include <cuda_runtime.h>
#define T_VEC 100

//DEFINICIÓN KERNEL CUDA

__global__ void sumaVectores(int* v1_device, int* v2_device, int*suma_device) {
  // printf("Índice de hilo dentro del bloque: %d\n", threadIdx.x);
  // printf("Índice del bloque: %d\n", blockIdx.x);
  // printf("Tamaño del bloque (número de hilos): %d\n", blockDim.x);
  int threadInBlockIndex = threadIdx.x + blockIdx.x * blockDim.x;

  // Cada hilo suma un cuarto del vector
    for (int i = threadInBlockIndex; i < T_VEC; i += blockDim.x * gridDim.x) {
        suma_device[i] = v1_device[i] + v2_device[i];
    }
}

//FUNCIÓN DEL HOST

int main() {

  //1 - INICIALIZAR DATOS DEL HOST

  int v1_host[T_VEC], v2_host[T_VEC], suma_host[T_VEC];
  for (int i = 0; i < T_VEC; i++) {
    v1_host[i] = 1;
    v2_host[i] = 2;
  }

  //2 - DECLARAR Y ASIGNAR MEMORIA PARA EL DEVICE

  int *v1_device, *v2_device, *suma_device; //punteros al device
  cudaMalloc((void**)&v1_device, sizeof(int) * T_VEC); //asignar memoria para el device
  cudaMalloc((void**)&v2_device, sizeof(int) * T_VEC);
  cudaMalloc((void**)&suma_device, sizeof(int) * T_VEC);

  //3 - TRANSFERIR DATOS DEL HOST AL DEVICE

  cudaMemcpy(v1_device, v1_host, sizeof(int) * T_VEC,cudaMemcpyHostToDevice); //transferir el valor del host al device
  cudaMemcpy(v2_device, v2_host, sizeof(int) * T_VEC,cudaMemcpyHostToDevice);
  //cudaMemcpy(suma_device, suma_host, sizeof(int) * T_VEC,cudaMemcpyHostToDevice); //Inutil

  //4 - EJECUTAR EN UNO O MÁS KERNELS

  sumaVectores<<<1, 25>>>(v1_device, v2_device, suma_device);
  //lanzar el kernel
  cudaDeviceSynchronize(); //espera hasta que el kernel termine

  //5 - TRANSFERIR DATOS DEL DEVICE AL HOST

  cudaMemcpy(suma_host, suma_device, sizeof(int) * T_VEC,
  cudaMemcpyDeviceToHost); //transferir el valor del device al host

  //6 - LIBERAR RECURSOS
  cudaFree(v1_device); //liberar memoria del device
  cudaFree(v2_device);
  cudaFree(suma_device);

  //MOSTRAR RESULTADO
  for (int i = 0; i < T_VEC; i++) {
    std::cout << suma_host[i] << " ";
  }
  std::cout << "\n";
 return 0;
}

3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 

